In [89]:
!pip install pyyaml h5py  

# House Pricing Prediction System

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,LeakyRelu
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback
# from category_encoder import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

ImportError: cannot import name 'LeakyRelu' from 'tensorflow.keras.layers' (C:\Users\HP\Downloads\anaconda new\Lib\site-packages\keras\_tf_keras\keras\layers\__init__.py)

In [38]:
df = pd.read_csv("C:/Users/HP/Downloads/Housing.csv")
df.head(10)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
5,10850000,7500,3,3,1,yes,no,yes,no,yes,2,yes,semi-furnished
6,10150000,8580,4,3,4,yes,no,no,no,yes,2,yes,semi-furnished
7,10150000,16200,5,3,2,yes,no,no,no,no,0,no,unfurnished
8,9870000,8100,4,1,2,yes,yes,yes,no,yes,2,yes,furnished
9,9800000,5750,3,2,4,yes,yes,no,no,yes,1,yes,unfurnished


In [39]:
 target = 'price'
X = df.drop(target,axis=1)
y = df[target]


In [12]:
Categorial_columns = [ 'mainroad','guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

In [40]:
Numeriacl_column = ['furnishingstatus','parking	']

In [41]:
df = pd.get_dummies(df,columns = ['furnishingstatus','parking'], drop_first=False)
df.head(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,prefarea,furnishingstatus_furnished,furnishingstatus_semi-furnished,furnishingstatus_unfurnished,parking_0,parking_1,parking_2,parking_3
0,13300000,7420,4,2,3,yes,no,no,no,yes,yes,True,False,False,False,False,True,False
1,12250000,8960,4,4,4,yes,no,no,no,yes,no,True,False,False,False,False,False,True
2,12250000,9960,3,2,2,yes,no,yes,no,no,yes,False,True,False,False,False,True,False
3,12215000,7500,4,2,2,yes,no,yes,no,yes,yes,True,False,False,False,False,False,True
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,no,True,False,False,False,False,True,False


In [42]:
# column = ['furnishingstatus','parking	']

In [45]:
columns_to_transform = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Apply the transformation to each column in the list
df[columns_to_transform] = df[columns_to_transform].applymap(lambda x: 1 if x == 'yes' else 0)

df.head(5)


C:\Users\HP\AppData\Local\Temp\ipykernel_5728\3199855502.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_transform] = df[columns_to_transform].applymap(lambda x: 1 if x == 'yes' else 0)


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,prefarea,furnishingstatus_furnished,furnishingstatus_semi-furnished,furnishingstatus_unfurnished,parking_0,parking_1,parking_2,parking_3
0,13300000,7420,4,2,3,1,0,0,0,1,1,True,False,False,False,False,True,False
1,12250000,8960,4,4,4,1,0,0,0,1,0,True,False,False,False,False,False,True
2,12250000,9960,3,2,2,1,0,1,0,0,1,False,True,False,False,False,True,False
3,12215000,7500,4,2,2,1,0,1,0,1,1,True,False,False,False,False,False,True
4,11410000,7420,4,1,2,1,1,1,0,1,0,True,False,False,False,False,True,False


## Spliting

In [69]:
target = 'price'
X = df.drop(target,axis=1)
y = df[target]


In [70]:
# splitting the data set into x test, x train, y test y train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [71]:

X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
y_train = np.array(y_train).astype('float32')  # or int, depending on your model's output layer
y_test = np.array(y_test).astype('float32')    # or int, depending on your model's output layer


In [72]:

# Creating the model
model  = LinearRegression()
model.fit(X_train,y_train)


# Print R2 and MSE for training and test sets
# # 3. Predict the target variable for both the training and test data
print(f" Score for Linear regresion training data {model.score(X_train,y_train)}" )
print(f" Score for Linear regresion testing data {model.score(X_test,y_test)}" )

print('*******************************************************')

# # Compute  MSE for the test data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate metrics
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

train_mse = mean_squared_error(y_train,y_train_pred)
test_mse = mean_squared_error(y_test,y_test_pred)

# Printing MSE for the train data
print("Training Data Mean Squared Error (MSE): (MSE:", train_mse)

# Printing MSE for the test data
print("Testing Data Mean Squared Error(MSE):", test_mse)

print('*******************************************************')

# Printing mae for train and test data
print("Mean Absolute Error (MAE) - Train:", mae_train)
print("Mean Absolute Error (MAE) - Test:", mae_test)


 Score for Linear regresion training data 0.6863347410348521
 Score for Linear regresion testing data 0.6558454329913128
*******************************************************
Training Data Mean Squared Error (MSE): (MSE: 967153100000.0
Testing Data Mean Squared Error(MSE): 1739553500000.0
*******************************************************
Mean Absolute Error (MAE) - Train: 718757.75
Mean Absolute Error (MAE) - Test: 963340.2


In [78]:
y_train.shape

(436,)

## Model

In [74]:
model = Sequential()
model.add(Dense(units = 24,kernel_initializer = 'uniform',activation = 'relu', input_dim = X_train.shape[1] ))
model.add(Dense(units =12 ,kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform',activation = 'linear'))
model.compile(optimizer = 'adam',loss = 'mean_squared_error',metrics=['mae'])

C:\Users\HP\Downloads\anaconda new\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [75]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 24)                  │             432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 12)                  │             300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │              13 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 745 (2.91 KB)

 Trainable params: 745 (2.91 KB)

 Non-trainable params: 0 (0.00 B)

## History

In [76]:
history = model.fit(X_train,y_train, epochs = 50 ,batch_size = 16, validation_data = (X_test,y_test))

Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - loss: 26036043513856.0000 - mae: 4784160.5000 - val_loss: 30129910710272.0000 - val_mae: 5007529.0000
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 24819875708928.0000 - mae: 4668127.5000 - val_loss: 30129239621632.0000 - val_mae: 5007468.5000
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 24867787243520.0000 - mae: 4695100.0000 - val_loss: 30126590918656.0000 - val_mae: 5007228.5000
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 24644690116608.0000 - mae: 4641013.0000 - val_loss: 30119884226560.0000 - val_mae: 5006619.5000
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 25366334799872.0000 - mae: 4724818.5000 - val_loss: 30106693140480.0000 - val_mae: 5005423.5000
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 25220509335552.0000 - mae: 4681503.0000 - val_loss: 30084310237184.0000 - val_mae: 5003392.5000
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 25458896797

# Tune Model

In [80]:
tune_model = Sequential()
tune_model.add(Dense(units = 64,kernel_initializer = 'uniform',activation = 'relu', input_dim = X_train.shape[1] ))
tune_model.add(Dense(units =48 ,kernel_initializer = 'uniform', activation = 'relu'))
tune_model.add(Dense(units =32 ,kernel_initializer = 'uniform', activation = 'relu'))
tune_model.add(Dense(units =16 ,kernel_initializer = 'uniform', activation = 'relu'))
tune_model.add(Dense(units = 1, kernel_initializer = 'uniform',activation = 'linear'))
tune_model.compile(optimizer = 'adam',loss = 'mean_squared_error',metrics=['mae'])

In [81]:
tune_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                     │ (None, 64)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 48)                  │           3,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 32)                  │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,385 (24.94 KB)

 Trainable params: 6,385 (24.94 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
from keras.callbacks import EarlyStopping
# Early Stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.01, restore_best_weights=True)
historyy = tune_model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 2864918036480.0000 - mae: 1225854.0000 - val_loss: 4040332935168.0000 - val_mae: 1564514.3750
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3427680649216.0000 - mae: 1348613.2500 - val_loss: 3925315420160.0000 - val_mae: 1541679.0000
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3024785506304.0000 - mae: 1258128.3750 - val_loss: 3994327711744.0000 - val_mae: 1555528.3750
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2534282625024.0000 - mae: 1194352.0000 - val_loss: 3996951248896.0000 - val_mae: 1556015.7500
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3175627620352.0000 - mae: 1312646.5000 - val_loss: 4000264749056.0000 - val_mae: 1556632.5000
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2920504885248.0000 - mae: 1254373.5000 - val_loss: 3967004966912.0000 - val_mae: 1550011.0000
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 3608010817536.00

# Save Model

In [90]:
# Save the entire model as a `.keras` zip archive.
tune_model.save('my_model_Housing_price_prediction.keras')